# Commands

In [ ]:
# oomstopper --no-tail tfidf-cache ; killbill tfidf-cache ; cd ~/twinews-logs ; jupython -o nohup-tfidf-cache-$HOSTNAME.out --venv st-venv ~/Workspace/Python/Datasets/Twinews/twinews/models/tfidf-cache.ipynb

# Init

In [ ]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
isNotebook = '__file__' not in locals()

In [ ]:
TEST = isNotebook # isNotebook, True, False

In [ ]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from nlptools.preprocessing import *
from nlptools.basics import *
from twinews.utils import *
from twinews.models.ranking import *

In [ ]:
from nltk.stem import WordNetLemmatizer 
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import LdaMulticore
from gensim.test.utils import common_corpus, common_dictionary
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
import gensim

In [ ]:
logger = Logger(tmpDir('logs') + "/tfidf-cache.log") if isNotebook else Logger("tfidf-cache-" + getHostname() + ".log")
tt = TicToc(logger=logger)
tt.tic()

# Config

In [ ]:
config = \
{
    'maxDocuments': 300 if TEST else 300000,
    'useExtraNews': False if TEST else True, # None = unlimited, 0 = no extra news
    'minDF': 1 / 500 if TEST else 1 / 2000, # Remove words that have a document frequency ratio lower than 1 / 500
    'maxDF': 300, # Remove top 300 voc elements
    
    'lowercase': True if TEST else True,
    'doLemmatization': False if TEST else False,
}

# Getting data

In [ ]:
# We get urls for the LDA model:
newsCollection = getNewsCollection()
urlsForModel = shuffle(list(newsCollection.distinct('url')), seed=1)
urlsForModel = urlsForModel[:config['maxDocuments']]

In [ ]:
# We get sentences:
sentences = getNewsSentences(urlsForModel, logger=logger)
bp(sentences, logger)

In [ ]:
# We flatten sentences:
for i in range(len(sentences)):
    sentences[i] = flattenLists(sentences[i])
docs = sentences
bp(docs, logger)

In [ ]:
# Lower case:
if config['lowercase']:
    for i in pb(list(range(len(docs))), logger=logger, message="Lower casing"):
        for u in range(len(docs[i])):
            docs[i][u] = docs[i][u].lower()
bp(docs, logger)

In [ ]:
# Lemmatization:
if config['doLemmatization']:
    lemmatizer = WordNetLemmatizer()
    pbar = ProgressBar(len(docs), logger=logger, message="Lemmatization")
    for i in range(len(docs)):
        for u in range(len(docs[i])):
            docs[i][u] = lemmatizer.lemmatize(docs[i][u])
        pbar.tic()
bp(docs, logger)

In [ ]:
# Filtering the corpus:
docs = filterCorpus(docs, minDF=config['minDF'], maxDF=config['maxDF'],
                    removeEmptyDocs=False, allowEmptyDocs=False, logger=logger)
for doc in docs: assert len(doc) > 0
bp(docs, logger)

In [ ]:
tt.tic("Data preprocessed")

# Infering topic vectors

In [ ]:
vectorizer = TfidfVectorizer\
(
    sublinear_tf=True,
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    # lowercase=True, # Doesn't work because we erased preprocessor
)
vectorizer.fit(docs)

In [ ]:
tt.tic("Model fitted")

# Infering and caching data

In [ ]:
from twinews.models.genericutils import getGenericCache, genericFields
if TEST:
    cache = getGenericCache("tfidf-test")
else:
    cache = getGenericCache("tfidf")
field = genericFields['tfidf']

In [ ]:
ids = list(newsCollection.distinct("_id"))

In [ ]:
if TEST:
    ids = ids[:1000]

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
for currentId in pb(ids, logger=logger, printRatio=0.01, verbose=not TEST):
    row = newsCollection.findOne({"_id": currentId}, projection={field: True})
    sentences = row[field]
    theHash = objectToHash(sentences)
    # We flatten sentences:
    doc = flattenLists(sentences)
    # We lowercase:
    if config['lowercase']:
        for i in range(len(doc)):
            doc[i] = doc[i].lower()
    # We lemmatize:
    if config['doLemmatization']:
        for i in range(len(doc)):
            doc[i] = lemmatizer.lemmatize(doc[i])
    # We vectorize it:
    vector = vectorizer.transform(np.array([doc]))
    # We print the doc:
    if TEST:
        bp(doc, logger)
        log(theHash, logger)
        log(vector, logger=logger)
    # We cache it:
    cache[theHash] = vector